In [15]:
import tushare as ts
import pandas as pd
import datetime
import logging

In [16]:
#basics = ts.get_stock_basics()
#basics.to_csv('e:/quant/data/basics.csv')
def getStockBaseMarketTime(code=600030):
    basics = pd.read_csv('d:/dev/quant/data/basics.csv')
    data = basics.loc[basics.code==code]
    timeToMarket = data['timeToMarket']
    startDate = datetime.datetime(timeToMarket/10000 ,timeToMarket%10000/100 , timeToMarket%10000%100) 
    return startDate.strftime('%Y-%m-%d')

#第一次从网络上获取然后存储在本地，后续均从本地文件获取
#内部调用函数
def getStockHistInfoInner(code=600030):
    startTime = getStockBaseMarketTime(code)
    print startTime 
    try:
        data = pd.read_csv('d:/dev/quant/data/'+str(code)+'.csv')
        data = data.sort_values(by="date" , ascending=True)
    except IOError as e:
        dt = datetime.datetime.now()   
        data = ts.get_h_data(str(code) , start=startTime , end=dt.strftime('%Y-%m-%d'))
        data.to_csv('d:/dev/quant/data/'+str(code)+'.csv')
        return None
    return data

#获取对应代码的股票历史行情信息
def getStockHistInfo(code=600030):
    data = getStockHistInfoInner(code)
    if data is None:
        data = getStockHistInfoInner(code)
    if data is None:
        logging.exception('the stock is still none')      
    return data
        
    
#EMA（12）= 前一日EMA（12）×11/13＋今日收盘价×2/13
#EMA（26）= 前一日EMA（26）×25/27＋今日收盘价×2/27
#DIFF=今日EMA（12）- 今日EMA（26）
#DEA（MACD）= 前一日DEA×8/10＋今日DIF×2/10 
#BAR=2×(DIFF－DEA)

def calcMacd(data=None , code=600030):
    if data is None:
        return None
    data['ema12'] = 0
    data['ema26'] = 0
    data['diff'] = 0
    data['dea'] = 0
    data['bar'] = 0
    
    
    #上市第一天数据值均为0
    data.iloc[0]['ema12'] = 0
    data.iloc[0]['ema26'] = 0
    data.iloc[0]['diff'] = 0
    data.iloc[0]['dea'] = 0
    data.iloc[0]['bar'] = 0
    
    
    for i in range(1,10,1):
        print type(data.iloc[i])
        print data.iloc[i].set_value('ema12' , round(round(data.iloc[i-1]['ema12'] * 11.0 / 13.0 , 4) + round(data.iloc[i]['close'] * 2.0 / 13.0 , 4) , 3))
        #print data.iloc[i]['ema12']
        data.iloc[i]['ema26'] = round(round(data.iloc[i-1]['ema26'] * 25.0 / 27.0 , 4) + round(data.iloc[i]['close'] * 2.0 / 27.0 , 4) , 3)
        data.iloc[i]['diff'] = round(data.iloc[i-1]['ema12'] - data.iloc[i-1]['ema26'] , 3)
        data.iloc[i]['dea'] = round(round(data.iloc[i-1]['dea'] * 8.0 / 10.0 , 4) + round(data.iloc[i]['diff'] * 2.0 / 10.0 , 4) , 3)
        data.iloc[i]['bar'] = round(2 * (data.iloc[i]['diff'] - data.iloc[i]['dea']) , 3)
    return data

data = getStockHistInfo(600030)
print data.head()
macd = calcMacd(data , 600030)
print data.head()
print macd.head()




D:\Program Files (x86)\anaconda\lib\site-packages\ipykernel\__main__.py:52: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Program Files (x86)\anaconda\lib\site-packages\ipykernel\__main__.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Program Files (x86)\anaconda\lib\site-packages\ipykernel\__main__.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Program Files (x86)\anaconda\lib\site-packages\ipykernel\__main__.py:55: SettingWithCopyWarning: 
A value is trying to be set o

2003-01-06
            date  open  high  close   low     volume      amount
3100  2003-01-06  1.10  1.11   1.00  0.99  194288556   998892578
3099  2003-01-07  0.99  1.01   0.97  0.96   58977717   290622430
3098  2003-01-08  0.96  1.06   1.06  0.96   72064078   368411113
3097  2003-01-09  1.12  1.17   1.17  1.12  110086868   636358017
3096  2003-01-10  1.23  1.29   1.24  1.22  206393422  1300495478
<class 'pandas.core.series.Series'>

D:\Program Files (x86)\anaconda\lib\site-packages\ipykernel\__main__.py:56: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Program Files (x86)\anaconda\lib\site-packages\ipykernel\__main__.py:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Program Files (x86)\anaconda\lib\site-packages\ipykernel\__main__.py:64: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



date       2003-01-07
open             0.99
high             1.01
close            0.97
low              0.96
volume    5.89777e+07
amount    2.90622e+08
ema12           0.149
ema26               0
diff                0
dea                 0
bar                 0
Name: 3099, dtype: object
<class 'pandas.core.series.Series'>
date       2003-01-08
open             0.96
high             1.06
close            1.06
low              0.96
volume    7.20641e+07
amount    3.68411e+08
ema12           0.163
ema26               0
diff                0
dea                 0
bar                 0
Name: 3098, dtype: object
<class 'pandas.core.series.Series'>
date       2003-01-09
open             1.12
high             1.17
close            1.17
low              1.12
volume    1.10087e+08
amount    6.36358e+08
ema12            0.18
ema26               0
diff                0
dea                 0
bar                 0
Name: 3097, dtype: object
<class 'pandas.core.series.Series'>
date       2003-01-10

D:\Program Files (x86)\anaconda\lib\site-packages\ipykernel\__main__.py:65: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
D:\Program Files (x86)\anaconda\lib\site-packages\ipykernel\__main__.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
